#Processo seletivo 2RPNet - Desafio

Validação, separação e armazenamento dos dados 

Devido à dificuldade que tive com o equipamento para realizar o desafio, usei apenas o dataset referente à fevereiro de 2023, ao invés dos últimos 3(fevereiro, janeiro e dezembro), como pedido. Onde necessário deixei comentado código que realizaria o pedido usando os 3 datasets

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
#spark
##iniciado o spark para analisar os dados

23/05/20 16:53:30 WARN Utils: Your hostname, guilherme-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/05/20 16:53:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/20 16:53:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('EPD_202302.csv',header=True, inferSchema=True)
##lendo e atribuindo o dataset. Caso os 3 datasets
## fossem ser utilizados, mais dois dataframes seriam
## criados(df2 e df3), e depois concatenados(dados que os schemas
## são idênticos) com
#df4 = df.union(df2)
#df-final = df4.union(df3)

##Analisando o dataframe

In [4]:
df.limit(5).toPandas()

23/05/20 17:00:01 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,YEAR_MONTH,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,ICB_NAME,ICB_CODE,PCO_NAME,PCO_CODE,PRACTICE_NAME,PRACTICE_CODE,ADDRESS_1,...,BNF_CODE,BNF_DESCRIPTION,BNF_CHAPTER_PLUS_CODE,QUANTITY,ITEMS,TOTAL_QUANTITY,ADQUSAGE,NIC,ACTUAL_COST,UNIDENTIFIED
0,202302,SOUTH WEST,Y58,NHS CORNWALL AND THE ISLES OF SCILLY INT,QT6,NHS CORNWALL AND THE ISLES OF SCILLY ICB,11N00,BOSVENA HEALTH,L82010,STILLMOOR HOUSE MED PRACT,...,0407020A0AAAUAU,Fentanyl 12micrograms/hour transdermal patches,04: Central Nervous System,5.0,3,15.0,0.0,37.77,35.44497,N
1,202302,SOUTH WEST,Y58,NHS CORNWALL AND THE ISLES OF SCILLY INT,QT6,NHS CORNWALL AND THE ISLES OF SCILLY ICB,11N00,BOSVENA HEALTH,L82010,STILLMOOR HOUSE MED PRACT,...,0407020A0AAAUAU,Fentanyl 12micrograms/hour transdermal patches,04: Central Nervous System,10.0,1,10.0,0.0,25.18,23.61758,N
2,202302,SOUTH WEST,Y58,NHS CORNWALL AND THE ISLES OF SCILLY INT,QT6,NHS CORNWALL AND THE ISLES OF SCILLY ICB,11N00,BOSVENA HEALTH,L82010,STILLMOOR HOUSE MED PRACT,...,0407020A0BBAIAE,Durogesic DTrans 25micrograms/hour transdermal...,04: Central Nervous System,10.0,1,10.0,0.0,35.98,33.74212,N
3,202302,SOUTH WEST,Y58,NHS CORNWALL AND THE ISLES OF SCILLY INT,QT6,NHS CORNWALL AND THE ISLES OF SCILLY ICB,11N00,BOSVENA HEALTH,L82010,STILLMOOR HOUSE MED PRACT,...,0407020A0BBAJAF,Durogesic DTrans 50micrograms/hour transdermal...,04: Central Nervous System,10.0,2,20.0,60.0,134.64,126.24407,N
4,202302,SOUTH WEST,Y58,NHS CORNWALL AND THE ISLES OF SCILLY INT,QT6,NHS CORNWALL AND THE ISLES OF SCILLY ICB,11N00,BOSVENA HEALTH,L82010,STILLMOOR HOUSE MED PRACT,...,0407020A0BQACAF,Fencino 50micrograms/hour transdermal patches,04: Central Nervous System,5.0,2,10.0,30.0,45.24,42.43537,N


In [5]:
df.printSchema()

root
 |-- YEAR_MONTH: integer (nullable = true)
 |-- REGIONAL_OFFICE_NAME: string (nullable = true)
 |-- REGIONAL_OFFICE_CODE: string (nullable = true)
 |-- ICB_NAME: string (nullable = true)
 |-- ICB_CODE: string (nullable = true)
 |-- PCO_NAME: string (nullable = true)
 |-- PCO_CODE: string (nullable = true)
 |-- PRACTICE_NAME: string (nullable = true)
 |-- PRACTICE_CODE: string (nullable = true)
 |-- ADDRESS_1: string (nullable = true)
 |-- ADDRESS_2: string (nullable = true)
 |-- ADDRESS_3: string (nullable = true)
 |-- ADDRESS_4: string (nullable = true)
 |-- POSTCODE: string (nullable = true)
 |-- BNF_CHEMICAL_SUBSTANCE: string (nullable = true)
 |-- CHEMICAL_SUBSTANCE_BNF_DESCR: string (nullable = true)
 |-- BNF_CODE: string (nullable = true)
 |-- BNF_DESCRIPTION: string (nullable = true)
 |-- BNF_CHAPTER_PLUS_CODE: string (nullable = true)
 |-- QUANTITY: double (nullable = true)
 |-- ITEMS: integer (nullable = true)
 |-- TOTAL_QUANTITY: double (nullable = true)
 |-- ADQUSAGE: d

In [7]:
df.count()

17238532

##As informações do dataset referentes aos "prescribers", ou seja, pessoas ou organizações que prescreveram receitas, são representadas por todas as colunas desde a primeira("REGIONAL_OFFICE_NAME") até a coluna "POST_CODE". A coluna mais relevante é a "PCO_NAME", que equivale ao nome da clínica, hospital ou semelhantes.

In [6]:
df.filter(df.PCO_NAME.isNull()).show()
##Como nenhum valor dessa coluna é nulo,
##  um dataframe vazio foi retornado

+----------+--------------------+--------------------+--------+--------+--------+--------+-------------+-------------+---------+---------+---------+---------+--------+----------------------+----------------------------+--------+---------------+---------------------+--------+-----+--------------+--------+---+-----------+------------+
|YEAR_MONTH|REGIONAL_OFFICE_NAME|REGIONAL_OFFICE_CODE|ICB_NAME|ICB_CODE|PCO_NAME|PCO_CODE|PRACTICE_NAME|PRACTICE_CODE|ADDRESS_1|ADDRESS_2|ADDRESS_3|ADDRESS_4|POSTCODE|BNF_CHEMICAL_SUBSTANCE|CHEMICAL_SUBSTANCE_BNF_DESCR|BNF_CODE|BNF_DESCRIPTION|BNF_CHAPTER_PLUS_CODE|QUANTITY|ITEMS|TOTAL_QUANTITY|ADQUSAGE|NIC|ACTUAL_COST|UNIDENTIFIED|
+----------+--------------------+--------------------+--------+--------+--------+--------+-------------+-------------+---------+---------+---------+---------+--------+----------------------+----------------------------+--------+---------------+---------------------+--------+-----+--------------+--------+---+-----------+---------

In [8]:
prescribers = df.select('YEAR_MONTH', 'REGIONAL_OFFICE_NAME', 'REGIONAL_OFFICE_CODE', 'ICB_NAME', 'ICB_CODE', 'PCO_NAME', 'PCO_CODE', 'PRACTICE_NAME', 'PRACTICE_CODE', 'ADDRESS_1', 'ADDRESS_3', 'ADDRESS_4', 'POSTCODE')

In [9]:
prescribers.printSchema()

root
 |-- YEAR_MONTH: integer (nullable = true)
 |-- REGIONAL_OFFICE_NAME: string (nullable = true)
 |-- REGIONAL_OFFICE_CODE: string (nullable = true)
 |-- ICB_NAME: string (nullable = true)
 |-- ICB_CODE: string (nullable = true)
 |-- PCO_NAME: string (nullable = true)
 |-- PCO_CODE: string (nullable = true)
 |-- PRACTICE_NAME: string (nullable = true)
 |-- PRACTICE_CODE: string (nullable = true)
 |-- ADDRESS_1: string (nullable = true)
 |-- ADDRESS_3: string (nullable = true)
 |-- ADDRESS_4: string (nullable = true)
 |-- POSTCODE: string (nullable = true)



In [10]:
prescribers.limit(5).toPandas()

,YEAR_MONTH,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,ICB_NAME,ICB_CODE,PCO_NAME,PCO_CODE,PRACTICE_NAME,PRACTICE_CODE,ADDRESS_1,ADDRESS_3,ADDRESS_4,POSTCODE
0,202302,SOUTH WEST,Y58,NHS CORNWALL AND THE ISLES OF SCILLY INT,QT6,NHS CORNWALL AND THE ISLES OF SCILLY ICB,11N00,BOSVENA HEALTH,L82010,STILLMOOR HOUSE MED PRACT,BODMIN,CORNWALL,PL31 2JJ
1,202302,SOUTH WEST,Y58,NHS CORNWALL AND THE ISLES OF SCILLY INT,QT6,NHS CORNWALL AND THE ISLES OF SCILLY ICB,11N00,BOSVENA HEALTH,L82010,STILLMOOR HOUSE MED PRACT,BODMIN,CORNWALL,PL31 2JJ
2,202302,SOUTH WEST,Y58,NHS CORNWALL AND THE ISLES OF SCILLY INT,QT6,NHS CORNWALL AND THE ISLES OF SCILLY ICB,11N00,BOSVENA HEALTH,L82010,STILLMOOR HOUSE MED PRACT,BODMIN,CORNWALL,PL31 2JJ
3,202302,SOUTH WEST,Y58,NHS CORNWALL AND THE ISLES OF SCILLY INT,QT6,NHS CORNWALL AND THE ISLES OF SCILLY ICB,11N00,BOSVENA HEALTH,L82010,STILLMOOR HOUSE MED PRACT,BODMIN,CORNWALL,PL31 2JJ
4,202302,SOUTH WEST,Y58,NHS CORNWALL AND THE ISLES OF SCILLY INT,QT6,NHS CORNWALL AND THE ISLES OF SCILLY ICB,11N00,BOSVENA HEALTH,L82010,STILLMOOR HOUSE MED PRACT,BODMIN,CORNWALL,PL31 2JJ


In [11]:
prescribers.count()
##possui o mesmo número de linhas que o df original,
## portanto, o novo df possui todos os prescribers
## da fonte

17238532

In [12]:
prescribers.write.parquet("prescribers.parquet")
##Escolhi salvar em arquivos parquet para ser
##mais rápido e de menor tamanho, mas salvar em 
## csv seria semelhante

##As informações do dataset referentes as "prescriptions", ou seja, receitas escritas por prescribers, são representadas por todas as colunas desde a "BNF_CHEMICAL_SUBSTANCE" até a coluna "UNIDENTIFIED". A coluna mais relevante é a "BNF_DESCRIPTION", que descreve informações como tipo, dosagem e nome comum da droga receitada

In [14]:
df.filter(df.BNF_DESCRIPTION.isNull()).show()
##Como nenhum valor dessa coluna é nulo,
##  um dataframe vazio foi retornado

+----------+--------------------+--------------------+--------+--------+--------+--------+-------------+-------------+---------+---------+---------+---------+--------+----------------------+----------------------------+--------+---------------+---------------------+--------+-----+--------------+--------+---+-----------+------------+
|YEAR_MONTH|REGIONAL_OFFICE_NAME|REGIONAL_OFFICE_CODE|ICB_NAME|ICB_CODE|PCO_NAME|PCO_CODE|PRACTICE_NAME|PRACTICE_CODE|ADDRESS_1|ADDRESS_2|ADDRESS_3|ADDRESS_4|POSTCODE|BNF_CHEMICAL_SUBSTANCE|CHEMICAL_SUBSTANCE_BNF_DESCR|BNF_CODE|BNF_DESCRIPTION|BNF_CHAPTER_PLUS_CODE|QUANTITY|ITEMS|TOTAL_QUANTITY|ADQUSAGE|NIC|ACTUAL_COST|UNIDENTIFIED|
+----------+--------------------+--------------------+--------+--------+--------+--------+-------------+-------------+---------+---------+---------+---------+--------+----------------------+----------------------------+--------+---------------+---------------------+--------+-----+--------------+--------+---+-----------+---------

In [13]:
prescriptions = df.select('YEAR_MONTH', 'BNF_CHEMICAL_SUBSTANCE', 'CHEMICAL_SUBSTANCE_BNF_DESCR', 'BNF_CODE', 'BNF_DESCRIPTION', 'BNF_CHAPTER_PLUS_CODE', 'QUANTITY', 'ITEMS', 'TOTAL_QUANTITY', 'ADQUSAGE', 'NIC', 'ACTUAL_COST', 'UNIDENTIFIED')

In [15]:
prescriptions.printSchema()

root
 |-- YEAR_MONTH: integer (nullable = true)
 |-- BNF_CHEMICAL_SUBSTANCE: string (nullable = true)
 |-- CHEMICAL_SUBSTANCE_BNF_DESCR: string (nullable = true)
 |-- BNF_CODE: string (nullable = true)
 |-- BNF_DESCRIPTION: string (nullable = true)
 |-- BNF_CHAPTER_PLUS_CODE: string (nullable = true)
 |-- QUANTITY: double (nullable = true)
 |-- ITEMS: integer (nullable = true)
 |-- TOTAL_QUANTITY: double (nullable = true)
 |-- ADQUSAGE: double (nullable = true)
 |-- NIC: double (nullable = true)
 |-- ACTUAL_COST: double (nullable = true)
 |-- UNIDENTIFIED: string (nullable = true)



In [16]:
prescriptions.limit(5).toPandas()

,YEAR_MONTH,BNF_CHEMICAL_SUBSTANCE,CHEMICAL_SUBSTANCE_BNF_DESCR,BNF_CODE,BNF_DESCRIPTION,BNF_CHAPTER_PLUS_CODE,QUANTITY,ITEMS,TOTAL_QUANTITY,ADQUSAGE,NIC,ACTUAL_COST,UNIDENTIFIED
0,202302,0407020A0,Fentanyl,0407020A0AAAUAU,Fentanyl 12micrograms/hour transdermal patches,04: Central Nervous System,5.0,3,15.0,0.0,37.77,35.44497,N
1,202302,0407020A0,Fentanyl,0407020A0AAAUAU,Fentanyl 12micrograms/hour transdermal patches,04: Central Nervous System,10.0,1,10.0,0.0,25.18,23.61758,N
2,202302,0407020A0,Fentanyl,0407020A0BBAIAE,Durogesic DTrans 25micrograms/hour transdermal...,04: Central Nervous System,10.0,1,10.0,0.0,35.98,33.74212,N
3,202302,0407020A0,Fentanyl,0407020A0BBAJAF,Durogesic DTrans 50micrograms/hour transdermal...,04: Central Nervous System,10.0,2,20.0,60.0,134.64,126.24407,N
4,202302,0407020A0,Fentanyl,0407020A0BQACAF,Fencino 50micrograms/hour transdermal patches,04: Central Nervous System,5.0,2,10.0,30.0,45.24,42.43537,N


In [17]:
prescriptions.count()
##possui o mesmo número de linhas que o df original,
## portanto, o novo df possui todos as prescriptions
## da fonte

17238532

In [18]:
prescriptions.write.parquet("prescriptions.parquet")